# Full Search

## Load Bible Text

In [1]:
from data import read_bible

unv = read_bible('https://bible.fhl.net/public/dnstrunv.tgz')

## Tokenize

In [2]:
import jieba
import jieba.posseg as pseg

jieba.enable_paddle()

Paddle enabled successfully......


In [3]:
from hanziconv import HanziConv

unv['s_text'] = unv.text.apply(HanziConv.toSimplified)

In [4]:
unv['s_text_tk'] = unv.s_text.apply(lambda v: pseg.lcut(v, use_paddle=True))

## Search

In [5]:
from typing import Iterable

def highlight_occurances(text: str, keywords: Iterable[str]) -> str:
    for i, kw in enumerate(keywords):
        text = text.replace(kw, highlight(kw, i))
    return text

def highlight(text: str, color_code: int) -> str:
    return f'\x1b[6;30;4{color_code + 1}m{text}\x1b[0m'

test_data = ['first', 'second', 'thrid', 'forth', 'fifth', 'sixth', 'seventh']
print(highlight_occurances(', '.join(test_data), test_data))

first, second, thrid, forth, fifth, sixth, seventh


In [6]:
from search import sentence_similarity

searches = ['欢乐 祈祷', '挂虑 祈祷', '虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

for search_term in searches:
    print(f'Search for {search_term}:')
    search_tk = pseg.lcut(search_term, use_paddle=True)
    match_scores = {}
    for v, vers_tk in zip(unv.s_text, unv.s_text_tk):
        similarity = sentence_similarity(search_tk, vers_tk)
        match_kw = [kw for kw in similarity.keys() if similarity[kw] > 0]
        vers = highlight_occurances(v, match_kw)
        score = sum(similarity.values())
        match_scores[vers] = score
    for top_match in sorted(match_scores, key=match_scores.get, reverse=True)[:10]:
        print(f'Match: {match_scores[top_match]:7.4f} Verse: {top_match}')
    print()

Search for 欢乐 祈祷:
Match:  1.0000 Verse: 我的神啊，因你启示仆人说，我必为你建立家室，所以仆人大胆在你面前祈祷。
Match:  1.0000 Verse: 大卫祷告神说：「我行这事大有罪了！现在求你除掉仆人的罪孽，因我所行的甚是愚昧。」
Match:  1.0000 Verse: 大卫祷告神说：「吩咐数点百姓的不是我吗？我犯了罪，行了恶，但这群羊做了甚么呢？愿耶和华─我神的手攻击我和我的父家，不要攻击你的民，降瘟疫与他们。」
Match:  1.0000 Verse: 夫妻不可彼此亏负，除非两相情愿，暂时分房，为要专心祷告方可；以后仍要同房，免得撒但趁着你们情不自禁，引诱你们。
Match:  1.0000 Verse: 凡男人祷告或是讲道（或译：说预言；下同），若蒙着头，就羞辱自己的头。
Match:  1.0000 Verse: 凡女人祷告或是讲道，若不蒙着头，就羞辱自己的头，因为这就如同剃了头发一样。
Match:  1.0000 Verse: 你们自己审察，女人祷告神，不蒙着头是合宜的吗？
Match:  1.0000 Verse: 我若用方言祷告，是我的灵祷告，但我的悟性没有果效。
Match:  1.0000 Verse: 这却怎么样呢？我要用灵祷告，也要用悟性祷告；我要用灵歌唱，也要用悟性歌唱。
Match:  1.0000 Verse: 惟求耶和华─我的神垂顾仆人的祷告祈求，俯听仆人今日在你面前的祈祷唿吁。

Search for 挂虑 祈祷:
Match:  2.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉神。
Match:  1.1429 Verse: 我愿你们无所挂虑。没有娶妻的，是为主的事挂虑，想怎样叫主喜悦。
Match:  1.1429 Verse: 娶了妻的，是为世上的事挂虑，想怎样叫妻子喜悦。
Match:  1.1429 Verse: 妇人和处女也有分别。没有出嫁的，是为主的事挂虑，要身体、灵魂都圣洁；已经出嫁的，是为世上的事挂虑，想怎样叫丈夫喜悦。
Match:  1.1429 Verse: 他必像树栽于水旁，在河边扎根，炎热来到，并不惧怕，叶子仍必青翠，在干旱之年毫无挂虑，而且结果不止。
Match:  1.0000 Verse: 我的神啊，因你启